# <center>Web Scraping</center>

### 1. Loading Dependencies

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import requests
from bs4 import BeautifulSoup 
import numpy as np
import pandas as pd
import re
from decimal import Decimal
import csv
import os
import string
import nltk
import ast
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet
#import math
#import matplotlib.pyplot as plt

### 2. Searching the drugs by category
We are using the link from WebMD to get top 5 drugs for the specified category. Here we have used "Chronic Pain" as our category and have obtained top 5 drugs after applying some optimization techniques.

In [2]:
page=requests.get("https://www.webmd.com/drugs/2/condition-3090/chronic%20pain") #Searching drugs by condition
soup=BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

In [3]:
#print(containers[0])
containers = soup.findAll("table", {"class": "drugs-treatments-table"})   #getting the name and review number of drugs
c = containers[0]

### 3. Collecting the links to obtain drug ID's names and the number of reviews.

In [4]:
links=c.find_all("a",href=re.compile("drugreview"))    #using 'drugview' substring to avoid repetition of links
links[0]
for link in links:
    print(link.get('href'))                            

<a href="/drugs/drugreview-4398-tramadol-hcl.aspx?drugid=4398&amp;drugname=tramadol-hcl">2905 Reviews</a>

/drugs/drugreview-4398-tramadol-hcl.aspx?drugid=4398&drugname=tramadol-hcl
/drugs/drugreview-4398-tramadol-hcl-er.aspx?drugid=4398&drugname=tramadol-hcl-er
/drugs/drugreview-1025-oxycodone-hcl.aspx?drugid=1025&drugname=oxycodone-hcl
/drugs/drugreview-1025-oxycodone-hcl-concentrate.aspx?drugid=1025&drugname=oxycodone-hcl-concentrate
/drugs/drugreview-1025-oxycodone-hcl-er.aspx?drugid=1025&drugname=oxycodone-hcl-er
/drugs/drugreview-1025-oxycodone-tablet-only.aspx?drugid=1025&drugname=oxycodone-tablet-only
/drugs/drugreview-2798-oxycontin.aspx?drugid=2798&drugname=oxycontin
/drugs/drugreview-64741-suboxone-film-medicated.aspx?drugid=64741&drugname=suboxone-film-medicated
/drugs/drugreview-64741-suboxone-tablet.aspx?drugid=64741&drugname=suboxone-tablet
/drugs/drugreview-2671-methadone-hcl.aspx?drugid=2671&drugname=methadone-hcl
/drugs/drugreview-2671-methadone-hcl-tablet.aspx?drugid=2671&drugname=methadone-hcl-tablet
/drugs/drugreview-2671-methadone-hcl-tablet-soluble.aspx?drugid=2671&dr

In [5]:
st=[]
did=[]
for link in links:
    st=(link.get('href'))                            #Getting the drug-id 
    loc = st.find("drugid=") + 7
    temp = ""
    while(True):
        if st[loc]=="&":
            break
        else:
            temp = temp + st[loc]
            loc = loc + 1
    did.append(temp)
    
print(did)

['4398', '4398', '1025', '1025', '1025', '1025', '2798', '64741', '64741', '2671', '2671', '2671', '144617', '327', '327', '327', '327', '327', '1507', '1509', '32971', '94892', '75100', '155153', '156955', '64748', '152917', '92576', '144614', '144614', '171739', '167438', '9730', '9730', '9730', '64740', '64740', '1508', '4101', '152125', '165699', '11996', '164754', '152562', '152562', '5360', '170329', '387', '166704', '18450', '4105', '155016', '63814', '9143', '156872', '92515', '9480', '9480', '4149', '165341', '165341', '61838', '156918', '4876', '152909', '152569', '406', '170175', '13733', '61833', '163158', '57795', '171738', '4100', '166431', '94727', '170241', '78328', '155959', '148740', '13732', '13735', '977', '977', '61834', '61835', '1925', '171613', '154348', '148312', '170232', '94724', '22001', '9383', '165098', '166836', '173366', '170300', '10683']


In [6]:
#c.findAll('a')
drug=c.find_all("a", href=re.compile("details"))          #Getting drugnames
dlist=[]
for i in range(12):
    l=drug[i].text
    dlist.append(l)
#print(dlist)

In [7]:
rlist=[]
for i in range(12):                                       #Getting number of reviews for each drug
    review=c.find_all("a", href=re.compile("drugreview"))
    r=review[i].text.split(" ")
    rlist.append(r[0])
#print(rlist)


### 4. Creating a dataframe of top 5 drugs by eliminating duplicates

In [8]:
df = pd.DataFrame(                                       #Creating a df of drug name and number of reviews for that drug
    {'Drug': dlist,
     'Reviews': rlist,
     'Drug_ID' : did[0:12],
    })
#drug_list
drugsv = df.drop_duplicates(subset='Reviews', keep="first")       #Drop duplicates

In [10]:
drugsv.to_csv('/Users/grv/Downloads/result1.csv')

### 5. Function to Iterate over the strings to be replaced

In [11]:
def replaceMultiple(mainString, toBeReplaces, newString):
    # Iterate over the strings to be replaced
    for elem in toBeReplaces :
        # Check if string is in the main string
        if elem in mainString :
            # Replace the string
            mainString = mainString.replace(elem, newString)
    
    return  mainString

### 6. Scrapping
The data for the drug name is scrapped from the webpage. It includes the reviews, ratings in 3 different categories and comments. The scrapped data is written to the csv file that is saved on the local machine. We currently are working on a sample of first 1000 records for computational convenience.


In [14]:
pages = [str(i) for i in range(200)]
filename= "reviews6.csv"
f = open(filename, "w")
headers = "Reviewer Details, Effectiveness Rating, Ease_of_Use Rating, Satisfaction Rating, Comment\n"
f.write(headers)
for page in pages:
    url1 = 'https://www.webmd.com/drugs/drugreview-4398-tramadol+oral.aspx?drugid=4398&drugname=tramadol+oral&pageIndex='+page+'&sortby=3&conditionFilter=-1'
    page=requests.get(url1)
    soup=BeautifulSoup(page.content, 'html.parser')
    con1 = soup.findAll("div", {"class": "userPost"})
    con2 = con1[1]
    r_details = con2.p.text
    ratings = con2.findAll("p", {"class" : "inlineRating starRating"})
    r_effec = ratings[0].text
    r_ease = ratings[1].text
    r_satis = ratings[2].text
    comment = con2.findAll("p", {"class" : "comment"})
    r_comment = comment[1].text
    
    filename= "reviews6.csv"
    f = open(filename, "a")
    for con2 in con1:
        r_details = con2.p.text
        ratings = con2.findAll("p", {"class" : "inlineRating starRating"})
        r_effec = ratings[0].text
        r_ease = ratings[1].text
        r_satis = ratings[2].text
        comment = con2.findAll("p", {"class" : "comment"})
        r_comment = comment[1].text
        
        f.write(r_details.replace("," , "|") + "," + r_effec[-1] + "," + r_ease[-1] + "," + r_satis[-1] + ","  + 
                                replaceMultiple(r_comment,['\r\n', ','], "|") + "\n")
        
        
        
f.close()

89

1289

872

139

449

...

102

154

579

117

291